# Named Entity Recognition using sklearn-crfsuite

In this notebook we train a basic CRF model for Named Entity Recognition on CoNLL2002 data (following https://github.com/TeamHG-Memex/sklearn-crfsuite/blob/master/docs/CoNLL2002.ipynb) and check its weights to see what it learned.

To follow this tutorial you need NLTK > 3.x and sklearn-crfsuite Python packages. The tutorial uses Python 3.

In [1]:
import nltk
import sklearn_crfsuite
import eli5

## 1. Training data

CoNLL 2002 datasets contains a list of Spanish sentences, with Named Entities annotated. It uses [IOB2](https://en.wikipedia.org/wiki/Inside_Outside_Beginning) encoding. CoNLL 2002 data also provide POS tags.

In [2]:
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

## 2. Feature extraction

POS tags can be seen as pre-extracted features. Let's extract more features (word parts, simplified POS tags, lower/title/upper flags, features of nearby words) and convert them to sklear-crfsuite format - each sentence should be converted to a list of dicts. This is a very simple baseline; you certainly can do better.

In [3]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],        
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

This is how features extracted from a single token look like:

In [4]:
X_train[0][1]

{'+1:postag': 'NP',
 '+1:postag[:2]': 'NP',
 '+1:word.istitle()': True,
 '+1:word.isupper()': False,
 '+1:word.lower()': 'australia',
 '-1:postag': 'NP',
 '-1:postag[:2]': 'NP',
 '-1:word.istitle()': True,
 '-1:word.isupper()': False,
 '-1:word.lower()': 'melbourne',
 'bias': 1.0,
 'postag': 'Fpa',
 'postag[:2]': 'Fp',
 'word.isdigit()': False,
 'word.istitle()': False,
 'word.isupper()': False,
 'word.lower()': '(',
 'word[-3:]': '('}

## 3. Train a CRF model

Once we have features in a right format we can train a linear-chain CRF (Conditional Random Fields) model using sklearn_crfsuite.CRF:

In [5]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1, 
    c2=0.1, 
    max_iterations=20, 
    all_possible_transitions=False,
)
crf.fit(X_train, y_train);

## 4. Inspect model weights

CRFsuite CRF models use two kinds of features: state features and transition features. Let's check their weights 
using eli5.explain_weights:

In [6]:
eli5.explain_weights(crf, top=30)

Explanation(estimator="CRF(algorithm='lbfgs', all_possible_states=None,\n  all_possible_transitions=False, averaging=None, c=None, c1=0.1, c2=0.1,\n  calibration_candidates=None, calibration_eta=None,\n  calibration_max_trials=None, calibration_rate=None,\n  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,\n  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=20,\n  max_linesearch=None, min_freq=None, model_filename=None,\n  num_memories=None, pa_type=None, period=None, trainer_cls=None,\n  variance=None, verbose=False)", description=None, error=None, method='CRF', is_regression=False, targets=[TargetExplanation(target='O', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='postag[:2]:Fp', weight=4.4156219999999999, std=None), FeatureWeight(feature='BOS', weight=3.1164190000000001, std=None), FeatureWeight(feature='bias', weight=2.401478, std=None), FeatureWeight(feature='word[-3:]:,', weight=2.296824, std=None), FeatureWeight(feature='word.lower():,', weight=2.296824, std=None), FeatureWeight(feature='postag:Fc', weight=2.296824, std=None), FeatureWeight(feature='postag[:2]:Fc', weight=2.296824, std=None), FeatureWeight(feature='postag:CC', weight=2.1241910000000002, std=None), FeatureWeight(feature='postag[:2]:CC', weight=2.1241910000000002, std=None), FeatureWeight(feature='EOS', weight=1.9841359999999999, std=None), FeatureWeight(feature='word.lower():y', weight=1.8585259999999999, std=None), FeatureWeight(feature='postag:RG', weight=1.683543, std=None), FeatureWeight(feature='postag[:2]:RG', weight=1.683543, std=None), FeatureWeight(feature='postag:Fg', weight=1.6096330000000001, std=None), FeatureWeight(feature='word.lower():-', weight=1.6096330000000001, std=None), FeatureWeight(feature='postag[:2]:Fg', weight=1.6096330000000001, std=None), FeatureWeight(feature='word[-3:]:-', weight=1.6096330000000001, std=None), FeatureWeight(feature='word.lower():.', weight=1.5816730000000001, std=None), FeatureWeight(feature='word[-3:]:.', weight=1.5816730000000001, std=None), FeatureWeight(feature='postag:Fp', weight=1.5816730000000001, std=None), FeatureWeight(feature='word[-3:]:y', weight=1.372123, std=None), FeatureWeight(feature='postag:CS', weight=1.1869989999999999, std=None), FeatureWeight(feature='postag[:2]:CS', weight=1.1869989999999999, std=None), FeatureWeight(feature='postag:Fpa', weight=1.149505, std=None), FeatureWeight(feature='word.lower():(', weight=1.149505, std=None), FeatureWeight(feature='word[-3:]:(', weight=1.149505, std=None)], neg=[FeatureWeight(feature='word.istitle()', weight=-6.1664279999999998, std=None), FeatureWeight(feature='word.isupper()', weight=-3.7227579999999998, std=None), FeatureWeight(feature='postag[:2]:NP', weight=-2.1055250000000001, std=None), FeatureWeight(feature='postag:NP', weight=-2.1055250000000001, std=None)], pos_remaining=16444, neg_remaining=3771), proba=None, score=None, weighted_spans=None), TargetExplanation(target='B-LOC', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='word.istitle()', weight=2.5299119999999999, std=None), FeatureWeight(feature='-1:word.lower():en', weight=2.2236449999999999, std=None), FeatureWeight(feature='word[-3:]:rid', weight=0.90603900000000004, std=None), FeatureWeight(feature='word.lower():madrid', weight=0.90516399999999997, std=None), FeatureWeight(feature='word.lower():españa', weight=0.64616700000000005, std=None), FeatureWeight(feature='word[-3:]:ona', weight=0.63950899999999999, std=None), FeatureWeight(feature='word[-3:]:aña', weight=0.59459799999999996, std=None), FeatureWeight(feature='+1:postag[:2]:Fp', weight=0.59452199999999999, std=None), FeatureWeight(feature='word.lower():parís', weight=0.51471299999999998, std=None), FeatureWeight(feature='word[-3:]:rís', weight=0.51353400000000005, std=None), FeatureWeight(feature='word.lower():barcelona', weight=0.42439100000000002, std=None), FeatureWeight(feature='-1:postag:Fg', weight=0.41972599999999999, std=None), FeatureWeight(featu

Features don't use gazetteers, so model had to remember some geographic names from the training data, e.g. that España is a location.

Transition features make sense: at least model learned that I-ENITITY must follow B-ENTITY, and that some transitions are unlikely, e.g. it is not common to have location right after an organization name (I-LOC -> B-ORG has a large negative weight). 

We'd also expect that O -> I-ENTIRY transitions have large negative weights because they are impossible, but these transitions have zero weight, not negative weight; it can be a problem, and decrease quality. sklearn_crfsuite.CRF provides ``all_possible_transitions`` argument which allows model to learn weights for transitions which are not observed in training data. Let's check how does it affect the result:

In [7]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1, 
    c2=0.1, 
    max_iterations=20, 
    all_possible_transitions=True,
)
crf.fit(X_train, y_train);

In [8]:
eli5.explain_weights(crf, top=5)

Explanation(estimator="CRF(algorithm='lbfgs', all_possible_states=None,\n  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,\n  calibration_candidates=None, calibration_eta=None,\n  calibration_max_trials=None, calibration_rate=None,\n  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,\n  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=20,\n  max_linesearch=None, min_freq=None, model_filename=None,\n  num_memories=None, pa_type=None, period=None, trainer_cls=None,\n  variance=None, verbose=False)", description=None, error=None, method='CRF', is_regression=False, targets=[TargetExplanation(target='O', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='BOS', weight=4.9311759999999998, std=None), FeatureWeight(feature='postag[:2]:Fp', weight=3.7544029999999999, std=None), FeatureWeight(feature='bias', weight=3.5389469999999998, std=None)], neg=[FeatureWeight(feature='word.istitle()', weight=-7.025055, std=None), FeatureWeight(feature='word.isupper()', weight=-3.6846450000000002, std=None)], pos_remaining=15043, neg_remaining=3906), proba=None, score=None, weighted_spans=None), TargetExplanation(target='B-LOC', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='word.istitle()', weight=2.396801, std=None), FeatureWeight(feature='-1:word.lower():en', weight=2.146674, std=None)], neg=[FeatureWeight(feature='-1:word.istitle()', weight=-1.2732810000000001, std=None), FeatureWeight(feature='postag:SP', weight=-1.0803510000000001, std=None), FeatureWeight(feature='postag[:2]:SP', weight=-1.0803510000000001, std=None)], pos_remaining=2284, neg_remaining=433), proba=None, score=None, weighted_spans=None), TargetExplanation(target='I-LOC', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='-1:word.lower():de', weight=0.88161299999999998, std=None), FeatureWeight(feature='-1:word.istitle()', weight=0.78004200000000001, std=None), FeatureWeight(feature='word[-3:]:de', weight=0.71821400000000002, std=None), FeatureWeight(feature='word.lower():de', weight=0.71089899999999995, std=None)], neg=[FeatureWeight(feature='BOS', weight=-1.9652970000000001, std=None)], pos_remaining=1684, neg_remaining=268), proba=None, score=None, weighted_spans=None), TargetExplanation(target='B-MISC', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='word.isupper()', weight=2.016886, std=None), FeatureWeight(feature='word.istitle()', weight=0.60338800000000004, std=None)], neg=[FeatureWeight(feature='postag[:2]:SP', weight=-0.95881400000000006, std=None), FeatureWeight(feature='postag:SP', weight=-0.95881400000000006, std=None), FeatureWeight(feature='-1:word.istitle()', weight=-0.85022500000000001, std=None)], pos_remaining=2287, neg_remaining=337), proba=None, score=None, weighted_spans=None), TargetExplanation(target='I-MISC', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='-1:word.istitle()', weight=0.86354399999999998, std=None), FeatureWeight(feature='-1:word.lower():de', weight=0.61597299999999999, std=None), FeatureWeight(feature='+1:postag[:2]:Fe', weight=0.59063699999999997, std=None), FeatureWeight(feature='+1:postag:Fe', weight=0.59063699999999997, std=None), FeatureWeight(feature='+1:word.lower():"', weight=0.59063699999999997, std=None)], neg=[], pos_remaining=3684, neg_remaining=582), proba=None, score=None, weighted_spans=None), TargetExplanation(target='B-ORG', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='word.isupper()', weight=3.0405470000000001, std=None), FeatureWeight(feature='word.lower():efe', weight=2.9521510000000002, std=None), FeatureWeight(feature='word[-3:]:EFE', weight=1.8509659999999999, std=None)], neg=[FeatureWeight(feature='postag[:2]:SP', weight=-1.416425, std=None), FeatureWeight(feature='postag:SP', weight=-1.416425, std=None)], pos_remaining=3528, neg_remaining=622), proba=None, score=None, weighted_spans=None), TargetExplanation(target='I-ORG', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='-1:word

With `all_possible_transitions=True` CRF learned large negative weights for impossible transitions like O -> I-ORG.

## 5. Customization

The table above is large and kind of hard to inspect; eli5 provides several options to look only at a part of features. You can check only a subset of labels:

In [9]:
eli5.explain_weights(crf, top=10, targets=['O', 'B-ORG', 'I-ORG'])

Explanation(estimator="CRF(algorithm='lbfgs', all_possible_states=None,\n  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,\n  calibration_candidates=None, calibration_eta=None,\n  calibration_max_trials=None, calibration_rate=None,\n  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,\n  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=20,\n  max_linesearch=None, min_freq=None, model_filename=None,\n  num_memories=None, pa_type=None, period=None, trainer_cls=None,\n  variance=None, verbose=False)", description=None, error=None, method='CRF', is_regression=False, targets=[TargetExplanation(target='O', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='BOS', weight=4.9311759999999998, std=None), FeatureWeight(feature='postag[:2]:Fp', weight=3.7544029999999999, std=None), FeatureWeight(feature='bias', weight=3.5389469999999998, std=None), FeatureWeight(feature='word[-3:]:,', weight=2.32761, std=None), FeatureWeight(feature='postag[:2]:Fc', weight=2.32761, std=None), FeatureWeight(feature='postag:Fc', weight=2.32761, std=None), FeatureWeight(feature='word.lower():,', weight=2.32761, std=None)], neg=[FeatureWeight(feature='word.istitle()', weight=-7.025055, std=None), FeatureWeight(feature='word.isupper()', weight=-3.6846450000000002, std=None), FeatureWeight(feature='postag[:2]:NP', weight=-2.1874950000000002, std=None)], pos_remaining=15039, neg_remaining=3905), proba=None, score=None, weighted_spans=None), TargetExplanation(target='B-ORG', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='word.isupper()', weight=3.0405470000000001, std=None), FeatureWeight(feature='word.lower():efe', weight=2.9521510000000002, std=None), FeatureWeight(feature='word[-3:]:EFE', weight=1.8509659999999999, std=None), FeatureWeight(feature='word.lower():gobierno', weight=1.2784279999999999, std=None), FeatureWeight(feature='word[-3:]:rno', weight=1.0331589999999999, std=None), FeatureWeight(feature='word.istitle()', weight=1.0053049999999999, std=None), FeatureWeight(feature='-1:word.lower():del', weight=0.86356100000000002, std=None)], neg=[FeatureWeight(feature='postag:SP', weight=-1.416425, std=None), FeatureWeight(feature='postag[:2]:SP', weight=-1.416425, std=None), FeatureWeight(feature='-1:word.lower():en', weight=-0.84217799999999998, std=None)], pos_remaining=3524, neg_remaining=621), proba=None, score=None, weighted_spans=None), TargetExplanation(target='I-ORG', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='-1:word.lower():de', weight=1.1589849999999999, std=None), FeatureWeight(feature='-1:word.istitle()', weight=0.992838, std=None), FeatureWeight(feature='-1:postag[:2]:SP', weight=0.63656400000000002, std=None), FeatureWeight(feature='-1:postag:SP', weight=0.63656400000000002, std=None), FeatureWeight(feature='-1:word.lower():real', weight=0.569994, std=None), FeatureWeight(feature='word.istitle()', weight=0.54742299999999999, std=None)], neg=[FeatureWeight(feature='bias', weight=-1.290432, std=None), FeatureWeight(feature='-1:word.isupper()', weight=-0.533057, std=None), FeatureWeight(feature='postag[:2]:VM', weight=-0.50821400000000005, std=None), FeatureWeight(feature='postag:VMI', weight=-0.47959000000000002, std=None)], pos_remaining=3517, neg_remaining=676), proba=None, score=None, weighted_spans=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=TransitionFeatureWeights(class_names=('O', 'B-ORG', 'I-ORG'), coef=array([[ 2.732368,  1.360375, -6.190191],
       [ 0.419399, -1.130284,  5.39191 ],
       [-0.116547, -1.44218 ,  5.141234]])))

Another option is to check only some of the features - it helps to check if a feature function works as intended. For example, let's check how word shape features are used by model using ``feature_re`` argument:

In [10]:
eli5.explain_weights(crf, top=10, feature_re='^word\.is')

Explanation(estimator="CRF(algorithm='lbfgs', all_possible_states=None,\n  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,\n  calibration_candidates=None, calibration_eta=None,\n  calibration_max_trials=None, calibration_rate=None,\n  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,\n  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=20,\n  max_linesearch=None, min_freq=None, model_filename=None,\n  num_memories=None, pa_type=None, period=None, trainer_cls=None,\n  variance=None, verbose=False)", description=None, error=None, method='CRF', is_regression=False, targets=[TargetExplanation(target='O', feature_weights=FeatureWeights(pos=[], neg=[FeatureWeight(feature='word.istitle()', weight=-7.025055, std=None), FeatureWeight(feature='word.isupper()', weight=-3.6846450000000002, std=None)], pos_remaining=0, neg_remaining=0), proba=None, score=None, weighted_spans=None), TargetExplanation(target='B-LOC', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='word.istitle()', weight=2.396801, std=None), FeatureWeight(feature='word.isupper()', weight=0.098533999999999997, std=None)], neg=[FeatureWeight(feature='word.isdigit()', weight=-0.15213399999999999, std=None)], pos_remaining=0, neg_remaining=0), proba=None, score=None, weighted_spans=None), TargetExplanation(target='I-LOC', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='word.istitle()', weight=0.46023700000000001, std=None)], neg=[FeatureWeight(feature='word.isupper()', weight=-0.345418, std=None), FeatureWeight(feature='word.isdigit()', weight=-0.018495000000000001, std=None)], pos_remaining=0, neg_remaining=0), proba=None, score=None, weighted_spans=None), TargetExplanation(target='B-MISC', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='word.isupper()', weight=2.016886, std=None), FeatureWeight(feature='word.istitle()', weight=0.60338800000000004, std=None)], neg=[FeatureWeight(feature='word.isdigit()', weight=-0.012468, std=None)], pos_remaining=0, neg_remaining=0), proba=None, score=None, weighted_spans=None), TargetExplanation(target='I-MISC', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='word.isdigit()', weight=0.27059699999999998, std=None)], neg=[FeatureWeight(feature='word.istitle()', weight=-0.106071, std=None), FeatureWeight(feature='word.isupper()', weight=-0.072343000000000005, std=None)], pos_remaining=0, neg_remaining=0), proba=None, score=None, weighted_spans=None), TargetExplanation(target='B-ORG', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='word.isupper()', weight=3.0405470000000001, std=None), FeatureWeight(feature='word.istitle()', weight=1.0053049999999999, std=None)], neg=[FeatureWeight(feature='word.isdigit()', weight=-0.044213000000000002, std=None)], pos_remaining=0, neg_remaining=0), proba=None, score=None, weighted_spans=None), TargetExplanation(target='I-ORG', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='word.istitle()', weight=0.54742299999999999, std=None), FeatureWeight(feature='word.isdigit()', weight=0.013778, std=None)], neg=[FeatureWeight(feature='word.isupper()', weight=-0.011927, std=None)], pos_remaining=0, neg_remaining=0), proba=None, score=None, weighted_spans=None), TargetExplanation(target='B-PER', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='word.istitle()', weight=1.7573190000000001, std=None), FeatureWeight(feature='word.isupper()', weight=0.049919999999999999, std=None)], neg=[FeatureWeight(feature='word.isdigit()', weight=-0.12321799999999999, std=None)], pos_remaining=0, neg_remaining=0), proba=None, score=None, weighted_spans=None), TargetExplanation(target='I-PER', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='word.istitle()', weight=0.97590200000000005, std=None), FeatureWeight(feature='word.isupper()', weight=0.19347400000000001, std=None)], neg=[FeatureWeight(feature='word.isdigit()', weight=-0.10626099999999999, std=None)], pos_remaining=0, neg_remaining=0), proba

Looks fine - UPPERCASE and Titlecase words are likely to be entities of some kind.

## 6. Formatting in console

It is also possible to format the result as text (could be useful in console):

In [11]:
expl = eli5.explain_weights(crf, top=5, targets=['O', 'B-LOC', 'I-LOC'])
print(eli5.format_as_text(expl))

Explained as: CRF

Transition features:
            O    B-LOC    I-LOC
-----  ------  -------  -------
O       2.732    1.217   -4.675
B-LOC  -0.226   -0.091    3.378
I-LOC  -0.184   -0.585    2.404

y='O' top features
----------------------------
  +4.931  BOS               
  +3.754  postag[:2]:Fp     
  +3.539  bias              
       …  (15043 more positive features)
       …  (3906 more negative features)
  -3.685  word.isupper()    
  -7.025  word.istitle()    

y='B-LOC' top features
----------------------------
  +2.397  word.istitle()    
  +2.147  -1:word.lower():en
       …  (2284 more positive features)
       …  (433 more negative features)
  -1.080  postag[:2]:SP     
  -1.080  postag:SP         
  -1.273  -1:word.istitle() 

y='I-LOC' top features
----------------------------
  +0.882  -1:word.lower():de
  +0.780  -1:word.istitle() 
  +0.718  word[-3:]:de      
  +0.711  word.lower():de   
       …  (1684 more positive features)
       …  (268 more negative features)
